In [118]:
write_html = False 

In [119]:
#%%capture
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [120]:
iOption0 = 10

In [121]:
centralities = ["0-93%","0-20%","20-40%","40-60%","60-93%","40-93%"]
Opt_names = ["loose", "basic","tight"]
Hists_names = ["m_{ee}_stat","m_{ee}_syst","dca_stat","dca_syst","dca_{J/#Psi}_stat","dca_{J/#Psi}_syst"]
part_names = ["#pi^{0} #rightarrow ee#gamma","#phi #rightarrow ee","J/#Psi","c#bar{c}","b#bar{b}","#omega #rightarrow ee","QGP","#pi^{-}","#gamma #rightarrow ee"]

In [122]:
N_centr = 6
N_options = 6
N_pt = 1
colors=[2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kBlack,root.kCyan,root.kYellow,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="output/files/"
file_names=["m_ee_DCA_10.root","m_ee_DCA_12.root","m_ee_DCA_16.root"]
sim_file_path="../sim/output/Newembed/invmass/decomposition/"
sim_file_names=['dalitz_50M_v5.root', 'phi_50M_v3.root', 'jpsi_50M_v3.root', 'ccbar_50M_v2.root', 'bbbar_10M_v1.root', 'omega_10M_v2.root', 'thermal_25M_200_v0.root', 'bbbar_10M_v0.root']
hist_names = ["inv_mass_stat","inv_mass_syst","dca_stat_0","dca_syst_0","dca_stat_1","dca_syst_1"]
hist_select_3D_names = ["inv_mass_dca_fg1_6","inv_mass_dca_fg1_5","inv_mass_dca_fg1_6","inv_mass_dca_fg1_7","inv_mass_dca_fg1_8","inv_mass_dca_fg1_7",\
                        "inv_mass_dca_fg2_6","inv_mass_dca_fg2_5","inv_mass_dca_fg2_6","inv_mass_dca_fg2_7","inv_mass_dca_fg2_8","inv_mass_dca_fg2_7",\
                        "inv_mass_dca_fg4_6","inv_mass_dca_fg4_5","inv_mass_dca_fg4_6","inv_mass_dca_fg3_7","inv_mass_dca_fg3_8","inv_mass_dca_fg3_7",\
                        "inv_mass_dca_gen_6","inv_mass_dca_gen_5","inv_mass_dca_gen_6","inv_mass_dca_gen_7","inv_mass_dca_gen_8","inv_mass_dca_gen_7"]
N_type = len(hist_names)
N_hists = len(hist_select_3D_names)

In [123]:
dNdy = [41.4,0.421,0.759/1000.,0.12,0.01,3.65,10,0.133/1000]
dNdy = [95./257*42.2,0.421,0.759/1000.,0.12,0.01,3.65,10,0.133/1000]
Ncolls = [251,771,282.4,82.6,12.1,40]
Br_to_ee = [1.174e-2, 2.97e-4, 5.94e-2, 0.01, 0.01, 7.28e-5, 1e-2, 7.9e-3]
RAA = [[0.4,0.8,0.4,0.4,0.4,0.4,0.4,0.4],[0.3,0.6,0.3,0.3,0.3,0.3,0.3,0.3],[0.4,0.8,0.4,0.4,0.4,0.4,0.4,0.4],[0.8,1.2,0.8,0.8,0.8,0.8,0.8,0.8],[0.9,1.0,0.9,0.9,0.9,0.9,0.9,0.9],[0.9,1.1,0.8,0.9,0.9,0.9,0.9,0.9]]
dNdy_centr = [[dNdy[i]*Ncolls[j]/42.2*RAA[j][i]**0 for i in range(8)] for j in range(6)]

In [124]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for icentr in range(N_centr):
            hist_select_3D0.append(infile.Get(hist_names[itype]+f"_{icentr}{0}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

    infile.Close()

hists_sim = []

for iFile in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
        if ihist%6==0:hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}_{1}")
        if ihist%6==5:hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}_{0}")
    hists_sim.append(hists_sim_file)
    infile.Close()

In [125]:
c0 = root.TCanvas(f"c0",f"c0",1500,1400)
c0.Divide(3,2)

legends =[]
for icentr in range(0,N_centr):

    legends.append(root.TLegend(0.6,0.6,0.99,0.99,centralities[icentr]))
    for iOpt in range(0,3):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        if iOpt==0 :
            Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        hists_read[iOpt][0][icentr].SetMarkerColorAlpha(colors[iOpt],(10-1)/10.)
        hists_read[iOpt][0][icentr].SetLineColorAlpha  (colors[iOpt],(10-1)/10.)
        if iOpt==0 :hists_read[iOpt][0][icentr].Draw("")
        else: hists_read[iOpt][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_read[iOpt][0][icentr],Opt_names[iOpt],"pl")
        hists_read[iOpt][1][icentr].SetFillColorAlpha(colors[iOpt],0.4)
        hists_read[iOpt][1][icentr].Draw("same e2")
    
        #legends[-1].AddEntry(0,"","a")
    Format_Legend(legends[-1],0.08,12)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

In [142]:
c0 = root.TCanvas(f"c0",f"c0",1500,1400)
c0.Divide(3,2)
iOpt = 0
legends =[]
keffs = [[200e-6,200e-6,20000e-6,6e-10,1e-10,200e-6,3e-8],[200e-6,200e-6,20000e-6,2e-9,1e-10,200e-6,25e-8],[200e-6,200e-6,20000e-6,8e-10,1e-10,200e-6,60e-9],\
         [200e-6,200e-6,20000e-6,1e-10,1e-10,200e-6,12e-9],[200e-6,200e-6,20000e-6,3e-11,16e-11,200e-6,9e-10],[200e-6,200e-6,20000e-6,14e-11,1e-10,200e-6,7e-9]]
for icentr in range(0,N_centr):
    legends.append(root.TLegend(0.6,0.6,0.99,0.99,centralities[icentr]))
    c0.cd(icentr+1)
    root.gPad.SetLogy()
    Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,1,3,1,0.85,1.24)
    hists_read[iOpt][0][icentr].Draw("")
    hists_read[iOpt][1][icentr].SetFillColorAlpha(1,0.4)
    hists_read[iOpt][1][icentr].Draw("same e2")

    legends[-1].AddEntry(hists_read[iOpt][0][icentr],Opt_names[iOpt],"pl")

    sum_sim_proj = hists_sim[len(sim_file_names)-1][icentr].ProjectionY(hists_sim[len(sim_file_names)-1][icentr].GetName()+"_proj") 
    sum_sim_proj.Add(sum_sim_proj,-1)
    for isim in range(0,len(sim_file_names)-1):
        sim_proj = hists_sim[isim][icentr].ProjectionY(hists_sim[isim][icentr].GetName()+"_proj",1,50,1,25) 
        Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim])
        if isim==6: 
            sim_proj.Rebin(1)
        sim_proj.Scale(keffs[icentr][isim])
        if icentr==3:  print(f"{hists_sim[isim][icentr+18].Integral()/hists_sim[isim][icentr+18].GetEntries()/Br_to_ee[isim]*10:.2E},",f"{hists_sim[isim][icentr+18].Integral()*keffs[icentr][isim]/Br_to_ee[isim]/20:.2E}, {dNdy_centr[icentr][isim]:.2E}",\
                             f"{hists_sim[isim][icentr+18].Integral()*keffs[icentr][isim]/Br_to_ee[isim]/dNdy_centr[icentr][isim]*100/20:.0f}%, {part_names[isim]}",f"{keffs[icentr][isim]*hists_sim[isim][icentr+18].GetEntries()/200:.1f},")
        sum_sim_proj.Add(sim_proj,1)
        sim_proj.Draw("same H")
        legends[-1].AddEntry(sim_proj,part_names[isim],"l")

    Format_Graph(sum_sim_proj,21,0,colors[isim+1],3,colors[isim+1])
    sum_sim_proj.Draw("same H")
    legends[-1].AddEntry(sum_sim_proj,"sum","l")
    #legends[-1].AddEntry(0,"","a")
    Format_Legend(legends[-1],0.08,12)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

1.28E+01, 1.40E+01, 3.05E+01 46%, #pi^{0} #rightarrow ee#gamma 1.1,
8.95E-01, 9.75E-01, 8.24E-01 118%, #phi #rightarrow ee 1.1,
5.25E-07, 5.73E-05, 1.49E-03 4%, J/#Psi 109.0,
9.86E+02, 5.27E-03, 2.35E-01 2%, c#bar{c} 0.0,
9.48E+02, 7.64E-04, 1.96E-02 4%, b#bar{b} 0.0,
3.90E+00, 4.25E+00, 7.14E+00 59%, #omega #rightarrow ee 1.1,
1.00E+03, 3.27E-01, 1.96E+01 2%, QGP 0.0,


In [132]:
c0 = root.TCanvas(f"c0",f"c0",1500,1400)
c0.Divide(3,2)
iOpt = 1
legends =[]
keffs = [[200e-6,200e-6,20000e-6,6e-10,1e-10,200e-6,3e-8],[200e-6,200e-6,20000e-6,2e-9,1e-10,200e-6,25e-8],[200e-6,200e-6,20000e-6,8e-10,1e-10,200e-6,60e-9],\
         [200e-6,200e-6,20000e-6,1e-10,1e-10,200e-6,12e-9],[200e-6,200e-6,20000e-6,3e-11,16e-11,200e-6,9e-10],[200e-6,200e-6,20000e-6,14e-11,1e-10,200e-6,7e-9]]
for icentr in range(0,N_centr):
    legends.append(root.TLegend(0.6,0.6,0.99,0.99,centralities[icentr]))
    c0.cd(icentr+1)
    root.gPad.SetLogy()
    Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,1,3,1,0.85,1.24)
    hists_read[iOpt][0][icentr].Draw("")
    hists_read[iOpt][1][icentr].SetFillColorAlpha(1,0.4)
    hists_read[iOpt][1][icentr].Draw("same e2")

    legends[-1].AddEntry(hists_read[iOpt][0][icentr],Opt_names[iOpt],"pl")

    sum_sim_proj = hists_sim[len(sim_file_names)-1][icentr].ProjectionY(hists_sim[len(sim_file_names)-1][icentr].GetName()+"_proj") 
    sum_sim_proj.Add(sum_sim_proj,-1)
    for isim in range(0,len(sim_file_names)-1):
        sim_proj = hists_sim[isim][icentr+6].ProjectionY(hists_sim[isim][icentr+6].GetName()+"_proj0",1,-1,1,-1) 
        Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim])
        sim_proj.Rebin(1)
        sim_proj.Scale(keffs[icentr][isim])
        sum_sim_proj.Add(sim_proj,1)
        sim_proj.Draw("same H")
        legends[-1].AddEntry(sim_proj,part_names[isim],"l")

    Format_Graph(sum_sim_proj,21,0,colors[isim+1],3,colors[isim+1])
    sum_sim_proj.Draw("same H")
    legends[-1].AddEntry(sum_sim_proj,"sum","l")
    #legends[-1].AddEntry(0,"","a")
    Format_Legend(legends[-1],0.08,12)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

In [133]:
c0 = root.TCanvas(f"c0",f"c0",1500,1400)
c0.Divide(3,2)
iOpt = 2
legends =[]
keffs = [[200e-6,200e-6,20000e-6,6e-10,1e-10,200e-6,3e-8],[200e-6,200e-6,20000e-6,2e-9,1e-10,200e-6,25e-8],[200e-6,200e-6,20000e-6,8e-10,1e-10,200e-6,60e-9],\
         [200e-6,200e-6,20000e-6,1e-10,1e-10,200e-6,12e-9],[200e-6,200e-6,20000e-6,3e-11,16e-11,200e-6,9e-10],[200e-6,200e-6,20000e-6,14e-11,1e-10,200e-6,7e-9]]
for icentr in range(0,N_centr):
    legends.append(root.TLegend(0.6,0.6,0.99,0.99,centralities[icentr]))
    c0.cd(icentr+1)
    root.gPad.SetLogy()
    Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,1,3,1,0.85,1.24)
    hists_read[iOpt][0][icentr].Draw("")
    hists_read[iOpt][1][icentr].SetFillColorAlpha(1,0.4)
    hists_read[iOpt][1][icentr].Draw("same e2")

    legends[-1].AddEntry(hists_read[iOpt][0][icentr],Opt_names[iOpt],"pl")

    sum_sim_proj = hists_sim[len(sim_file_names)-1][icentr].ProjectionY(hists_sim[len(sim_file_names)-1][icentr].GetName()+"_proj") 
    sum_sim_proj.Add(sum_sim_proj,-1)
    for isim in range(0,len(sim_file_names)-1):
        sim_proj = hists_sim[isim][icentr+12].ProjectionY(hists_sim[isim][icentr+12].GetName()+"_proj",1,-1,1,-1) 
        Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim])
        sim_proj.Rebin(1)
        sim_proj.Scale(keffs[icentr][isim])
        sum_sim_proj.Add(sim_proj,1)
        sim_proj.Draw("same H")
        legends[-1].AddEntry(sim_proj,part_names[isim],"l")

    Format_Graph(sum_sim_proj,21,0,colors[isim+1],3,colors[isim+1])
    sum_sim_proj.Draw("same H")
    legends[-1].AddEntry(sum_sim_proj,"sum","l")
    #legends[-1].AddEntry(0,"","a")
    Format_Legend(legends[-1],0.08,12)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

In [134]:
c0 = root.TCanvas(f"c0",f"c0",1400,1000)
c0.Divide(3,2)
sim_dca = [1,2,3,4,6]
keffs = [[200e-6,200e-6,20000e-6,6e-10,1e-10,200e-6,3e-8],[200e-6,200e-6,20000e-6,2e-9,1e-10,200e-6,25e-8],[200e-6,200e-6,20000e-6,8e-10,1e-10,200e-6,60e-9],\
         [200e-6,200e-6,20000e-6,1e-10,1e-10,200e-6,12e-9],[200e-6,200e-6,20000e-6,3e-11,16e-11,200e-6,9e-10],[200e-6,200e-6,20000e-6,14e-11,1e-10,200e-6,7e-9]]
legends =[]
iOpt = 1
for icentr in range(0,N_centr):

    legends.append(root.TLegend(0.4,0.75,0.98,0.98,centralities[icentr]))
    for ihist in range(2,3):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
        
        hists_read[iOpt][ihist][icentr].SetMarkerColorAlpha(colors[ihist],(10-1)/10.)
        hists_read[iOpt][ihist][icentr].SetLineColorAlpha  (colors[ihist],(10-1)/10.)
        hists_read[iOpt][ihist][icentr].Draw("")

        hists_read[iOpt][ihist+1][icentr].SetFillColorAlpha(colors[ihist],0.4)
        hists_read[iOpt][ihist+1][icentr].Draw("same e2")

        legends[-1].AddEntry(hists_read[iOpt][ihist][icentr],Hists_names[ihist]+" "+Opt_names[iOpt],"pl")
        

        sum_sim_proj = hists_sim[len(sim_file_names)-1][icentr].ProjectionX(hists_sim[len(sim_file_names)-1][icentr].GetName()+"_proj_dca") 
        sum_sim_proj.Add(sum_sim_proj,-1)
        for isim in sim_dca:
            sim_proj = hists_sim[isim][icentr+6].ProjectionX(hists_sim[isim][icentr+6].GetName()+"_proj_dca",22,55,1,25) 
            Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim])
            sim_proj.Scale(keffs[icentr][isim]/300)
            if icentr==3:  print(f"{hists_sim[isim][icentr+18].Integral()*keffs[icentr][isim]/Br_to_ee[isim]:.2E}, {dNdy_centr[icentr][isim]:.2E}",\
                                 f"{hists_sim[isim][icentr+18].Integral()*keffs[icentr][isim]/Br_to_ee[isim]/dNdy_centr[icentr][isim]*100:.0f}%, {part_names[isim]}")
            sum_sim_proj.Add(sim_proj,1)
            sim_proj.Draw("same H")
            legends[-1].AddEntry(sim_proj,part_names[isim],"l")
        
        Format_Legend(legends[-1],0.08)
        legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

1.95E+01, 8.24E-01 2367%, #phi #rightarrow ee
1.15E-03, 1.49E-03 77%, J/#Psi
1.05E-01, 2.35E-01 45%, c#bar{c}
1.53E-02, 1.96E-02 78%, b#bar{b}
6.54E+00, 1.96E+01 33%, QGP


In [130]:
c0 = root.TCanvas(f"c0",f"c0",1400,1000)
c0.Divide(3,2)

legends =[]
iOpt = 0
for icentr in range(0,N_centr):

    legends.append(root.TLegend(0.4,0.75,0.98,0.98,centralities[icentr]))
    for ihist in range(2,6,2):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        if ihist==2 :
            Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        hists_read[iOpt][ihist][icentr].SetMarkerColorAlpha(colors[ihist],(10-1)/10.)
        hists_read[iOpt][ihist][icentr].SetLineColorAlpha  (colors[ihist],(10-1)/10.)
        if ihist==2 :hists_read[iOpt][ihist][icentr].Draw("")
        else: hists_read[iOpt][ihist][icentr].Draw("same")
        hists_read[iOpt][ihist+1][icentr].SetFillColorAlpha(colors[ihist],0.4)
        hists_read[iOpt][ihist+1][icentr].Draw("same e2")
        legends[-1].AddEntry(hists_read[iOpt][ihist][icentr],Hists_names[ihist]+" "+Opt_names[iOpt],"pl")
    
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

In [131]:
c0 = root.TCanvas(f"c0",f"c0",1400,1000)
c0.Divide(3,2)

legends =[]
ihist = 2
N_loc_options = 5
for icentr in range(0,N_centr):

    legends.append(root.TLegend(0.6,0.6,0.99,0.99,centralities[icentr]))
    for iOpt in range(0,3):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        if iOpt==0 :
            Format_Hist_total(hists_read[iOpt][0][icentr],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        hists_read[iOpt][ihist][icentr].SetMarkerColorAlpha(colors[iOpt],(10-1)/10.)
        hists_read[iOpt][ihist][icentr].SetLineColorAlpha  (colors[iOpt],(10-1)/10.)
        if iOpt==0 :hists_read[iOpt][ihist][icentr].Draw("")
        else: hists_read[iOpt][ihist][icentr].Draw("same")
        hists_read[iOpt][ihist+1][icentr].SetFillColorAlpha(colors[iOpt],0.4)
        hists_read[iOpt][ihist+1][icentr].Draw("same e2")
        legends[-1].AddEntry(hists_read[iOpt][ihist][icentr],Opt_names[iOpt],"pl")
    
    for iOpt in range(1+N_loc_options,N_options):
        legends[-1].AddEntry(0,"","a")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")